In [1]:
import sys
sys.path.append('../src')

In [2]:
import torch
from transformer_wrappers.wrappers.context import ContextCausalLMWrapper
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [4]:
COLOURS = [f'C{i}' for i in range(10)]
STYLES = ['solid', 'dotted', 'dashed', 'dashdot']

In [5]:
import os
TOKEN = os.environ['HUGGINGFACE_TOKEN']

In [6]:
# MODEL = 'gpt2-xl'
# MODEL = 'mistralai/Mistral-7B-Instruct-v0.2'  
# MODEL = 'meta-llama/Llama-2-7b-hf'
MODEL = 'google/gemma-7b'
MODEL_CONFIGS = {
    'torch_dtype': torch.bfloat16,
    'attn_implementation': 'eager',
    'device_map': DEVICE,
    'token': TOKEN,
}

# TOKENIZER = 'gpt2-xl'
# TOKENIZER = 'mistralai/Mistral-7B-Instruct-v0.2'  
# TOKENIZER = 'meta-llama/Llama-2-7b-hf'
TOKENIZER = 'google/gemma-7b'
TOKENIZER_CONFIGS = {'token': TOKEN}

In [7]:
model = ContextCausalLMWrapper.from_pretrained(MODEL, 
                                               att_len=2,
                                               model_kwargs=MODEL_CONFIGS, 
                                               tokenizer_name_or_path=TOKENIZER, 
                                               tokenizer_kwargs=TOKENIZER_CONFIGS,
                                               )
model.enable_benchmarking()

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

ValueError: Please provide a value for att_len!

In [ ]:
input_string = 'The capital of Italy is'

In [ ]:
input_encodings = model.tokenizer(input_string, return_tensors='pt').to(DEVICE)

In [ ]:
output_wrapper = model(
    **input_encodings,
    return_dict=True,
    output_attentions=True,
    use_cache=True,
    output_hidden_stsates=True,
    return_attention_output=True,  # Self-attention layer output
    return_feed_forward_output=True
)

In [ ]:
output = model.generate(input_encodings.input_ids,
                        do_sample=False, 
                        max_length=25, 
                        return_dict_in_generate=True, 
                        output_attentions=True)

ValueError: The following `model_kwargs` are not used by the model: ['att_len'] (note: typos in the generate arguments will also show up in this list)

In [ ]:
model.tokenizer.decode(output['sequences'].squeeze())

'<bos>The capital of Italy is capital is is is is is is is is is is is is is is is is is is'

In [ ]:
output.attentions[8][0][0, 0, 0, :]

tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.6539e-04, 3.1738e-02,
        5.8203e-01, 3.8477e-01], device='cuda:0', dtype=torch.bfloat16)